In [90]:
import csv

In [91]:
with open('SIRENE_GEO_122018.csv','r', encoding='utf-8') as read_obj:
    csv_reader = csv.DictReader(read_obj, delimiter=';')
    k = 0
    print('\n')
    for row in csv_reader:
        print(row)
        k += 1
        if k == 2:
            break



{'\ufeffSIRET': '04007240700014', 'SIREN': '040072407', 'NIC': '00014', 'CODE_INSEE': '69088', 'LIBCOM': 'FONTAINES SUR SAONE', 'ADR_NORMAL': 'COPROPRIETE - 2 CHEMIN DE LA PETITE CERDAGNE - 69270 FONTAINES SUR SAONE - FRANCE', 'ADR_DECLAR': 'COPROPRIETE - 2 CHE DE LA PTE CERDAGNE - 69270 FONTAINES SUR SAONE - ', 'NJ': '9110', 'NJ_NIV1': 'Groupement de droit privé', 'NJ_NIV2': 'Syndicat de propriétaires', 'NJ_NIV3': 'Syndicat de copropriété ', 'SIEGE': '1', 'ENSEIGNE': '', 'IND_PUBLIPO': '0', 'DIFFCOM': 'O', 'AMINTRET': '201209', 'NATETAB': '', 'LIBNATETAB': '', 'APET700': '8110Z', 'NAFET_NIV1': 'Activités de services administratifs et de soutien', 'NAFET_NIV2': 'Services relatifs aux bâtiments et aménagement paysager', 'NAFET_NIV3': 'Activités combinées de soutien lié aux bâtiments', 'NAFET_NIV4': 'Activités combinées de soutien lié aux bâtiments', 'NAFET_NIV5': 'Activités combinées de soutien lié aux bâtiments', 'DAPET': '2008', 'TEFET': '01', 'LIBTEFET': '1 ou 2 salariés', 'EFETCEN

In [92]:
with open("interesting_naf_codes.txt", 'r') as f:
    k = 0
    for line in f:
        print(line)
        print(type(line))
        k += 1
        if k > 3:
            break
    
    k = 0
    for line in f:
        naf, defin = line.split(', ')
        print("NAF : ", naf)
        print("def : ", defin)
        print('\n')
        if k > 3:
            break

3511Z;/ production d’électricité

<class 'str'>
3512Z;/ transport d’électricité

<class 'str'>
3513Z;/ distribution d’électricité

<class 'str'>
3514Z;/ commerce d’électricité

<class 'str'>


ValueError: not enough values to unpack (expected 2, got 1)

## Création du geoJSON

In [93]:
mind_geo = dict(type="FeatureCollection")
mind_geo['features'] = []

## ouverture des codes NAF et définitions sous forme de liste

In [94]:
naf_codes = []
definitions = []

f = open("interesting_naf_codes.txt", 'r')
for line in f:
    print(line.split(';/ '))
    naf, defin = line.split(';/ ')
    naf_codes.append(naf)
    definitions.append(defin[:-1])

f.close()

print('nafs : ', naf_codes)
print('\n')
print('definitions', definitions)

['3511Z', 'production d’électricité\n']
['3512Z', 'transport d’électricité\n']
['3513Z', 'distribution d’électricité\n']
['3514Z', 'commerce d’électricité\n']
['3521Z', 'production de combustibles gazeux\n']
['3522Z', 'distribution de combustibles gazeux par conduite\n']
['3523Z', 'commerce de combustibles gazeux par conduite\n']
['3600Z', 'captage, traitement et distribution d’eau\n']
['3700Z', 'collecte et traitement des eaux usées\n']
['3811Z', 'collecte des déchets non dangereux\n']
['3812Z', 'collecte des déchets dangereux\n']
['3821Z', 'traitement et élimination des déchets non dangereux\n']
['3822Z', 'traitement et élimination des déchets dangereux\n']
['3832Z', 'récupération de déchets triés\n']
['3900Z', 'dépollution et autres services de gestion des déchets\n']
['3211Z', 'frappe de monnaie\n']
['3212Z', 'fabrication d’article de joaillerie et bijouterie\n']
['3213Z', 'fabrication d‘articles de bijouterie fantaisie et articles similaires\n']
['3220Z', 'fabrication d’instrument

# Ouverture du CSV géocodé, check si dans la liste NAF, création du geoJSON

In [95]:
with open('SIRENE_GEO_122018.csv','r', encoding='utf-8') as read_obj:
    csv_reader = csv.DictReader(read_obj, delimiter=';')
    header = next(csv_reader)
    k = 0

    for row in csv_reader:
        k += 1
        #if k > 1000:
            #break
            
        #tester si le code NAF est présent dans la base de données
        naf = row['APET700']
        #print("naf :", naf)
        
        if naf in naf_codes:
            i = naf_codes.index(naf)
            #print("Naf trouvé dans la liste: ", naf)
            feature = dict(type="Feature")
            feature["properties"] = {}

            #aller chercher les infos nécessaires dans le csv (adresse, NAF, )
            coords = [float(row['_lon_epsg4326']), float(row['_lat_epsg4326'])]
            print("coords : ", coords)
            adresse = row["ADR_NORMAL"]
            siret = row["\ufeffSIRET"]
            siren = row["SIREN"]
            effectif = row["LIBTEFET"]
            nom = 'rien'
            
            feature["properties"]['Adresse'] = adresse
            feature["properties"]["NAF"] = naf
            feature["properties"]["SIRET"] = siret
            feature["properties"]["SIREN"] = siren
            feature["properties"]["Effectif"] = effectif
            feature["properties"]["Activité"] = definitions[i]
            print("Activité : ", feature["properties"]["Activité"])
            feature["geometry"] = dict(type='Point')
            feature["geometry"]["coordinates"] = coords #aller chercher les coordonnées dans le csv
            if k%10 == 0:
                print("feature : ", feature)
            
            # ajout de la feature au geoJSON
            mind_geo["features"].append(feature)

print("\n fin ! ")
        

coords :  [4.831871925486115, 45.76935526259051]
Activité :  fabrication de vêtements de dessus
coords :  [4.868489545792376, 45.80497719694253]
Activité :  réparation de machines et équipements mécaniques
coords :  [4.964003878864947, 45.707899943639504]
Activité :  réparation de machines et équipements mécaniques
coords :  [4.909183307366412, 45.68291160237304]
Activité :  réparation d’équipements électroniques
coords :  [4.921068065089869, 45.72681125959561]
Activité :  fabrication de structures métalliques et de parties de structures
feature :  {'type': 'Feature', 'properties': {'Adresse': 'BAUDIN CHATEAUNEUF - 2 RUE MARYSE BASTIE - 69500 BRON - FRANCE', 'NAF': '2511Z', 'SIRET': '08578053400153', 'SIREN': '085780534', 'Effectif': 'Unités non employeuses', 'Activité': 'fabrication de structures métalliques et de parties de structures'}, 'geometry': {'type': 'Point', 'coordinates': [4.921068065089869, 45.72681125959561]}}
coords :  [4.960255280607975, 45.735822144659096]
Activité :  

coords :  [4.863467948049853, 45.82190196745828]
Activité :  boulangerie et boulangerie-pâtisserie
feature :  {'type': 'Feature', 'properties': {'Adresse': 'MONSIEUR YVES BREST - 1 ALLEE DE LA CHARDONNIERE - 69270 FONTAINES SUR SAONE - FRANCE', 'NAF': '1071C', 'SIRET': '39938408000046', 'SIREN': '399384080', 'Effectif': '3 à 5 salariés', 'Activité': 'boulangerie et boulangerie-pâtisserie'}, 'geometry': {'type': 'Point', 'coordinates': [4.863467948049853, 45.82190196745828]}}
coords :  [4.840228429179649, 45.722409343930885]
Activité :  fabrication de béton prêt à l’emploi
coords :  [4.940055808359959, 45.66489738158392]
Activité :  fabrication de béton prêt à l’emploi
coords :  [4.929810461480837, 45.732508845355994]
Activité :  fabrication de béton prêt à l’emploi
coords :  [4.8482396204655425, 45.67226281262792]
Activité :  fabrication de béton prêt à l’emploi
coords :  [4.925829480560618, 45.70637567381595]
Activité :  installation de structures métalliques chaudronnées et de tuyaut

Activité :  réparation de machines et équipements mécaniques
coords :  [4.811150666492993, 45.783103805155626]
Activité :  métallurgie des autres métaux non ferreux
coords :  [5.03055996798444, 45.771454003498285]
Activité :  fabrication de structures métalliques et de parties de structures
coords :  [4.8488730041014705, 45.75760382672165]
Activité :  fabrication de vêtements de dessus
coords :  [4.8213107741453305, 45.742322501706454]
Activité :  boulangerie et boulangerie-pâtisserie
coords :  [4.844429813206493, 45.76695139947371]
Activité :  boulangerie et boulangerie-pâtisserie
coords :  [4.820716775477123, 45.7436803291432]
Activité :  boulangerie et boulangerie-pâtisserie
coords :  [4.866551851026821, 45.744951710276744]
Activité :  boulangerie et boulangerie-pâtisserie
coords :  [4.920925708471792, 45.76049719899239]
Activité :  traitement et élimination des déchets dangereux
coords :  [4.804231958510599, 45.88397297579487]
Activité :  boulangerie et boulangerie-pâtisserie
coord

Activité :  fabrication de matériel médico-chirurgical et dentaire
coords :  [4.853101910429809, 45.753905759572035]
Activité :  commerce de combustibles gazeux par conduite
coords :  [4.862397692686126, 45.71732120929751]
Activité :  construction de véhicules automobiles
coords :  [4.955212335487084, 45.714333372137155]
Activité :  réparation de matériels électroniques et optiques
coords :  [4.842793968587453, 45.70704196588137]
Activité :  exploitation de gravières et sablières, extraction d’argiles et de kaolin
coords :  [4.938265402780964, 45.7751679188803]
Activité :  fabrication d’instrumentation scientifique et technique
coords :  [4.770679074830415, 45.800060143353235]
Activité :  extraction de pierres ornementales et de construction, de calcaire industriel, de gypse, de craie et d’ardoise
coords :  [4.869111036819253, 45.7623883489375]
Activité :  fabrication d‘articles de bijouterie fantaisie et articles similaires
coords :  [4.964225639023121, 45.70821446251429]
Activité :  

coords :  [4.883564414213484, 45.77491145274896]
Activité :  boulangerie et boulangerie-pâtisserie
coords :  [4.923030749234234, 45.67938639434405]
Activité :  fabrication d’engrenages et d’organes mécaniques de transmission
coords :  [4.828458364143104, 45.76312210083156]
Activité :  fabrication de vêtements de dessus
feature :  {'type': 'Feature', 'properties': {'Adresse': 'MADAME GUNILLA HOLM - UTOJ - 3 RUE DES TROIS MARIES - 69005 LYON 5EME - ', 'NAF': '1413Z', 'SIRET': '81990428500023', 'SIREN': '819904285', 'Effectif': '0 salarié', 'Activité': 'fabrication de vêtements de dessus'}, 'geometry': {'type': 'Point', 'coordinates': [4.828458364143104, 45.76312210083156]}}
coords :  [4.931993335919902, 45.76245919003141]
Activité :  fabrication d’engrenages et d’organes mécaniques de transmission
coords :  [5.058992638103277, 45.80384448042768]
Activité :  transformation et conservation de fruits
coords :  [4.833199057619518, 45.87452699173786]
Activité :  fabrication de matériel médico

...

coords :  [4.861758985454697, 45.716058307420944]
Activité :  production d’électricité
coords :  [4.861758985454697, 45.716058307420944]
Activité :  production d’électricité
coords :  [4.878028178577962, 45.768989488545195]
Activité :  fabrication de jeux et jouets
coords :  [4.857208231951912, 45.76929430647957]
Activité :  fabrication d‘articles de bijouterie fantaisie et articles similaires
coords :  [4.766587517640623, 45.59709906129524]
Activité :  installation de structures métalliques chaudronnées et de tuyauterie
coords :  [4.859350646312681, 45.723769669285765]
Activité :  fabrication d’objets divers en bois ; fabrication d’objets en liège, vannerie et sparterie
feature :  {'type': 'Feature', 'properties': {'Adresse': 'MONSIEUR PHILIPPE CROZE - 67 RUE DU PROFESSEUR ROUX - 69200 VENISSIEUX - FRANCE', 'NAF': '1629Z', 'SIRET': '83812614200010', 'SIREN': '838126142', 'Effectif': '0 salarié', 'Activité': 'fabrication d’objets divers en bois ; fabrication d’objets en liège, vannerie

coords :  [4.912898747957454, 45.725953905104134]
Activité :  traitement et revêtement des métaux
feature :  {'type': 'Feature', 'properties': {'Adresse': 'MONSIEUR AMINE AMMARI - 144 RAYOS - 190 AVENUE SAINT EXUPERY - 69500 BRON - FRANCE', 'NAF': '2561Z', 'SIRET': '83993789300015', 'SIREN': '839937893', 'Effectif': '0 salarié', 'Activité': 'traitement et revêtement des métaux'}, 'geometry': {'type': 'Point', 'coordinates': [4.912898747957454, 45.725953905104134]}}
coords :  [4.826071695783942, 45.78181129513821]
Activité :  fabrication d’autres textiles n.c.a.
coords :  [4.822194531465701, 45.74261462229329]
Activité :  fabrication d’articles céramiques à usage domestique ou ornemental
coords :  [4.813061413279617, 45.77501971112329]
Activité :  production d’électricité
coords :  [4.813061413279617, 45.77501971112329]
Activité :  production d’électricité
coords :  [4.8262437148781645, 45.76117313097755]
Activité :  ennoblissement textile
coords :  [4.780888082437453, 45.76408794916535

coords :  [4.795215925738501, 45.68494371461356]
Activité :  fabrication de matériel médico-chirurgical et dentaire
coords :  [4.7710038160304675, 45.73767940382953]
Activité :  autres activités manufacturières n.c.a
coords :  [4.845349933971781, 45.7625662421005]
Activité :  fabrication d’autres vêtements et accessoires
coords :  [4.854290408586324, 45.72251630832818]
Activité :  fabrication de vêtements de dessus
coords :  [4.9177968485173755, 45.748318944901236]
Activité :  fabrication de structures métalliques et de parties de structures
coords :  [4.861758985454697, 45.716058307420944]
Activité :  fabrication de bière
coords :  [5.002112637246513, 45.76728017676]
Activité :  pâtisserie
coords :  [4.881290105078728, 45.75425671024344]
Activité :  réparation et maintenance d’aéronefs et d’engins spatiaux
coords :  [4.8373251090787655, 45.77849821148002]
Activité :  fabrication de vêtements de dessus
coords :  [4.835872265630092, 45.766836438358474]
Activité :  fabrication de parfums

coords :  [4.93951750983258, 45.7514168256777]
Activité :  fabrication d’articles de brosserie
coords :  [4.873281287000081, 45.81674360392615]
Activité :  fabrication de vêtements de travail
feature :  {'type': 'Feature', 'properties': {'Adresse': 'ROBUR - 54 AVENUE DE L INDUSTRIE - 69140 RILLIEUX LA PAPE - FRANCE', 'NAF': '1412Z', 'SIRET': '95750213100031', 'SIREN': '957502131', 'Effectif': '20 à 49 salariés', 'Activité': 'fabrication de vêtements de travail'}, 'geometry': {'type': 'Point', 'coordinates': [4.873281287000081, 45.81674360392615]}}
coords :  [4.817488114224886, 45.88718219041517]
Activité :  fabrication de matériel médico-chirurgical et dentaire
coords :  [4.93622213292409, 45.75778887077395]
Activité :  fabrication de moteurs, génératrices et transformateurs électriques
coords :  [4.812825351360491, 45.69626195038792]
Activité :  fabrication de pâtes alimentaires
feature :  {'type': 'Feature', 'properties': {'Adresse': 'LUSTUCRU FRAIS - N 3 5 - 37 CHEMIN DES MOULINS - 

coords :  [4.822767481397848, 45.80924825040637]
Activité :  conception d’ensemble et assemblage sur site industriel et d’équipements de contrôle des processus industriels
coords :  [4.776602105541398, 45.73110204279006]
Activité :  fabrication de préparations pharmaceutiques
coords :  [4.756070507884185, 45.74910897059768]
Activité :  fabrication de préparations pharmaceutiques
coords :  [4.9114608044756185, 45.778139161974146]
Activité :  mécanique industrielle
coords :  [4.872353488376245, 45.80657607060304]
Activité :  découpage, emboutissage
coords :  [4.843887310464001, 45.75997980058929]
Activité :  fabrication de vêtements de dessus
coords :  [4.833048410106941, 45.85112513864055]
Activité :  autres activités manufacturières n.c.a
coords :  [4.836641819214232, 45.76719995758794]
Activité :  fabrication d’article de joaillerie et bijouterie
coords :  [4.834716998377819, 45.66677811169793]
Activité :  fabrication de plaques , feuilles, tubes et profilés en matières plastiques
coo

In [81]:
print(mind_geo)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'Adresse': 'MADAME CLAIRE DE LAGARDE - CLAIRE COUTURE - 122 MTE DE LA GRANDE COTE - 69001 LYON 1ER - ', 'NAF': '1413Z', 'SIRET': '05439629600037', 'SIREN': '054396296', 'Effectif': '0 salarié', 'Activité': 'fabrication de vêtements de dessus'}, 'geometry': {'type': 'Point', 'coordinates': [4.831871925486115, 45.76935526259051]}}, {'type': 'Feature', 'properties': {'Adresse': 'JF CESBRON - 14 22 - 14 AVENUE BARTHELEMY THIMONNIER - 69300 CALUIRE ET CUIRE - FRANCE', 'NAF': '3312Z', 'SIRET': '06620112000715', 'SIREN': '066201120', 'Effectif': '1 ou 2 salariés', 'Activité': 'réparation de machines et équipements mécaniques'}, 'geometry': {'type': 'Point', 'coordinates': [4.868489545792376, 45.80497719694253]}}, {'type': 'Feature', 'properties': {'Adresse': 'JF CESBRON - 36 RUE DU CHAMP DOLIN - 69800 ST PRIEST - FRANCE', 'NAF': '3312Z', 'SIRET': '06620112001135', 'SIREN': '066201120', 'Effectif': '10 à 19 salariés'

## Sauvegarde du geoJSON ainsi créé

In [96]:
## sauvegarde du geoJSON ainsi créé
import json

with open('mind_geo.json', 'w') as fp:
    json.dump(mind_geo, fp)